# Batch Convert ND2 Files to TIFF

Select directory containing ND2 files. Script will create new folder within directory called 'tiff', and will convert ND2 files to TIFF, then save in tiff folder.

### Install libraries (only need to do this first time)

In [ ]:
pip install nd2reader tifffile numpy


In [1]:
import os
from tkinter import Tk, filedialog
from nd2reader import ND2Reader
from tifffile import imwrite
import numpy as np

def convert_nd2_to_tiff(input_dir, output_dir):
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Iterate through ND2 files in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith(".nd2"):
            nd2_path = os.path.join(input_dir, filename)
            output_filename = os.path.splitext(filename)[0] + ".tiff"
            output_path = os.path.join(output_dir, output_filename)

            # Convert ND2 to TIFF based on user-provided conditions
            with ND2Reader(nd2_path) as images:
                num_channels = images.sizes['c']
                num_frames = images.sizes['t']
                num_slices = images.sizes.get('z', 1)
                image_shape = (num_frames, num_slices, num_channels, images.metadata['height'], images.metadata['width'])
                stack = np.zeros(image_shape, dtype=np.uint16)

                for t in range(num_frames):
                    for c in range(num_channels):
                        for z in range(num_slices):
                            # Rearrange the channel order (Z, C, T, Y, X)
                            stack[t, z, c, :, :] = images.get_frame_2D(t=t, z=z, c=c)

                # Save the stack as TIFF
                imwrite(output_path, stack, imagej=True)

def main():
    # Ask user to select input directory
    root = Tk()
    root.withdraw()  # Hide the main window
    input_dir = filedialog.askdirectory(title="Select Directory containing .nd2 files")
    root.destroy()

    if not input_dir:
        print("No directory selected. Exiting.")
        return

    # Define output directory
    output_dir = os.path.join(input_dir, "tiff")

    # Convert ND2 files to TIFF based on user-provided conditions
    convert_nd2_to_tiff(input_dir, output_dir)
    print("Conversion complete. TIFF files saved in:", output_dir)

if __name__ == "__main__":
    main()


Conversion complete. TIFF files saved in: C:/Users/oes6098/Desktop/test images nd2\tiff
